In [48]:
import pandas as pd
from sklearn.linear_model import LogisticRegression

In [49]:
df = pd.read_csv('GWAS_clinical.csv')
df.head()

,FamID,CAD,sex,age,tg,hdl,ldl
0,10002,1,1,60,NaN,NaN,NaN
1,10004,1,2,50,55.0,23.0,75.0
2,10005,1,1,55,105.0,37.0,69.0
3,10007,1,1,52,314.0,54.0,108.0
4,10008,1,1,58,161.0,40.0,94.0


In [118]:
log_reg = LogisticRegression(solver='lbfgs')

In [131]:
row_n = len(df.index)
train_rows_n = int(row_n * 0.3)

df_train = df.sample(train_rows_n).dropna()
df_train_data = df_train.loc[:, ['sex', 'tg', 'hdl', 'ldl']]
df_train_target = df_train.loc[:, 'CAD']

log_reg.fit(df_train_data, df_train_target)

LogisticRegression(C=0.5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [132]:
test_df = df.dropna()
test_data = test_df.loc[:, ['sex', 'tg', 'hdl', 'ldl']]
test_target = test_df.loc[:, 'CAD']
log_reg.score(test_data, test_target)

0.7080405932864949